In [1]:
!pip install mlflow boto3 awscli

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.2/548.2 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 443.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.1/575.1 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203

In [4]:
!aws configure

In [3]:
import mlflow

mlflow.set_tracking_uri("Your Tracking URI")

In [6]:
# Set or create an experiment
mlflow.set_experiment("Exp 3 - TfIdf Trigram new max_features")

2024/11/19 16:55:43 INFO mlflow.tracking.fluent: Experiment with name 'Exp 3 - TfIdf Trigram new max_features' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://interview-mlflow-bucket/531908695991360182', creation_time=1732035343809, experiment_id='531908695991360182', last_update_time=1732035343809, lifecycle_stage='active', name='Exp 3 - TfIdf Trigram new max_features', tags={}>

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report , confusion_matrix
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pandas as pd

In [8]:
df = pd.read_csv("https://raw.githubusercontent.com/ashishshukla888/influencer_sentiment_analysis/refs/heads/master/data.csv").dropna(subset=['clean_comment'])
df.head()
df.shape

(37149, 2)

In [9]:
# S1 :- Func to Run exp
def run_experiment_tfidf_maxfeature(max_features):
  ngram_range = (1,3) # trigram setting

  # S2 :- tfidf vectorizer with diff max feature
  vectorizer = TfidfVectorizer(ngram_range=ngram_range,max_features=max_features)

  # S3 :- split data
  X_train , X_test , y_train , y_test = train_test_split(df['clean_comment'],df['category'], test_size=0.2, random_state=42 , stratify = df['category'])

  X_train = vectorizer.fit_transform(X_train)
  X_test = vectorizer.transform(X_test)

  # S4 :- Define & train RF model
  with mlflow.start_run() as run:

    # define tag for experiment and the run
    mlflow.set_tag("mlflow.runName",f"TFIDF_Trigram_max_features_{max_features}")
    mlflow.set_tag("experiment_type","Feature_engineering")
    mlflow.set_tag("model_type","RandomForestClassifier")

    # Add a description
    mlflow.set_tag("description",f"Random_Forest with TFIDF_Trigram_max_features_{max_features}")

    # Log vectorizer params
    mlflow.log_param("vectorizer_type","TF-IDF")
    mlflow.log_param("ngram_range",ngram_range)
    mlflow.log_param("vectorizer_max_features",max_features)

    # Log the RF params

    n_estimators = 200
    max_depth = 15

    mlflow.log_param("n_estimators",n_estimators)
    mlflow.log_param("max_depth",max_depth)

    # Train the model

    model = RandomForestClassifier(n_estimators=n_estimators,max_depth=max_depth,random_state=42)
    model.fit(X_train,y_train)

    # make prediction
    y_pred = model.predict(X_test)

    # Log accuracy
    accuracy = accuracy_score(y_test,y_pred)
    mlflow.log_metric("accuracy", accuracy)

    # Log Classification report
    classification_rep = classification_report(y_test,y_pred, output_dict=True)
    for label,metrics in classification_rep.items():
      if isinstance(metrics,dict):
        for metric,value in metrics.items():
          mlflow.log_metric(f"{label}_{metric}",value)

    # Log confusion matrix
    conf_matrix = confusion_matrix(y_test,y_pred)
    plt.figure(figsize=(8,6))
    sns.heatmap(conf_matrix,annot=True,fmt="d",cmap="Blues")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(f"Confusion_Matrix: TF-IDF Trigrams, max_features{max_features}")
    plt.savefig("confusion_matrix.png")
    mlflow.log_artifact("confusion_matrix.png")
    plt.close()

    # log the model
    mlflow.sklearn.log_model(model,f"random_forest_model_tfidf_trigram_{max_features}")

# Test various max feature values
max_features_values = [1000,2000,3000,4000,5000,6000,7000,8000,9000,10000]

for max_features in max_features_values:
  run_experiment_tfidf_maxfeature(max_features)

2024/11/19 16:56:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TFIDF_Trigram_max_features_1000 at: http://ec2-35-174-3-91.compute-1.amazonaws.com:5000/#/experiments/531908695991360182/runs/ba1b3815a18c4f2ca8bb11090abfaca0
🧪 View experiment at: http://ec2-35-174-3-91.compute-1.amazonaws.com:5000/#/experiments/531908695991360182


2024/11/19 16:57:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TFIDF_Trigram_max_features_2000 at: http://ec2-35-174-3-91.compute-1.amazonaws.com:5000/#/experiments/531908695991360182/runs/09dc3874bdec449fb4ee1cb0ac188110
🧪 View experiment at: http://ec2-35-174-3-91.compute-1.amazonaws.com:5000/#/experiments/531908695991360182


2024/11/19 16:57:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run TFIDF_Trigram_max_features_8000 at: http://ec2-35-174-3-91.compute-1.amazonaws.com:5000/#/experiments/531908695991360182/runs/dee53a3e1e7644d4a7e58a3ebb637ca4
🧪 View experiment at: http://ec2-35-174-3-91.compute-1.amazonaws.com:5000/#/experiments/531908695991360182


2024/11/19 16:59:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TFIDF_Trigram_max_features_9000 at: http://ec2-35-174-3-91.compute-1.amazonaws.com:5000/#/experiments/531908695991360182/runs/957a2244bf8d454298ab4a4c8fef7687
🧪 View experiment at: http://ec2-35-174-3-91.compute-1.amazonaws.com:5000/#/experiments/531908695991360182


2024/11/19 17:00:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TFIDF_Trigram_max_features_10000 at: http://ec2-35-174-3-91.compute-1.amazonaws.com:5000/#/experiments/531908695991360182/runs/d86b4a77e24244ec956394672b34131a
🧪 View experiment at: http://ec2-35-174-3-91.compute-1.amazonaws.com:5000/#/experiments/531908695991360182
